<a href="https://colab.research.google.com/github/King-Rian/Project-3/blob/main/preprocessing_emoji_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Install emoji library
!pip install emoji
import emoji

In [6]:
# Retrieve the 'emojis.json' zippped file

from google.colab import files
import zipfile
import json
import pandas as pd

# Step 1: Upload the file
uploaded = files.upload()

# Step 2: Extract the ZIP file
with zipfile.ZipFile('emojis.json.zip', 'r') as zip_ref:
    zip_ref.extractall()

# Step 3: Load the extracted JSON file
emoji_file = 'emojis.json'

# Using pandas
emoji_df = pd.read_json(emoji_file)
print("Dataset loaded successfully. Sample data:")
print(emoji_df.head())


Saving emojis.json.zip to emojis.json (1).zip
Dataset loaded successfully. Sample data:
                                            category  \
0  Miscellaneous Symbols And Pictographs -> Emoji...   
1  Miscellaneous Symbols And Pictographs -> Emoji...   
2                                               None   
3  Miscellaneous Symbols And Pictographs -> Emoji...   
4                                               None   

                                            keywords  \
0  [dark skin tone, hand, forbidden, gesture, wom...   
1                     [dark skin tone, woman, guard]   
2                 [racing, running, woman, marathon]   
3  [gymnastics, medium-light skin tone, woman, ca...   
4                                      [woman, golf]   

                                          definition  \
0  The Woman Gesturing Not OK, Type-6 emoji is a ...   
1  The Female Guard, Type-6 emoji is a sequence o...   
2  The female version of the ?? Runner emoji. The...   
3  The Woman D

In [7]:
# Check the shape, features, and datatypes of the df
emoji_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2389 entries, 0 to 2388
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   category    1988 non-null   object
 1   keywords    2389 non-null   object
 2   definition  2389 non-null   object
 3   unicode     2389 non-null   object
 4   name        2389 non-null   object
 5   shortcode   845 non-null    object
 6   senses      2389 non-null   object
dtypes: object(7)
memory usage: 130.8+ KB


In [8]:
# Set options to display all rows and columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_columns', None)  # Display all columns

# Create a new emojis DataFrame with 'name' and 'unicode'
key_value_pairs_df = emoji_df[['name', 'unicode']]

# Process the 'name' column to include : at the beginning and end of the emoji name
key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
    lambda x: f":{x.split(':')[0].strip()}:"  # Remove everything after the first colon and add colons
)

# Convert the unicode column to emoji symbols
key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)

# Filter out rows where 'unicode' contains more than one Unicode value
# We check for the number of spaces and ensure it is less than 1
# Eliminating complex emojis and ZWJ sequence values
filtered_df = key_value_pairs_df[key_value_pairs_df['unicode'].str.count(' ') < 1]

# Remove rows where 'emoji' contains text with a beginning and ending ':' - which means that emoji was not found for that :name:
filtered_df = key_value_pairs_df[~key_value_pairs_df['emoji'].str.match(r'^:.*:$')]

# Create a new df and rename features to read for concatination with the next data set
new_filtered_df = filtered_df[['name', 'emoji']]
new_filtered_df = new_filtered_df.rename(columns={'name': 'label'})

# Display the DataFrame with emoji symbols
new_filtered_df.head()

<ipython-input-8-5759284d03f7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['name'] = key_value_pairs_df['name'].apply(
<ipython-input-8-5759284d03f7>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  key_value_pairs_df['emoji'] = key_value_pairs_df['name'].apply(emoji.emojize)


,label,emoji
15,:squid:,🦑
16,:shrimp:,🦐
17,:rhinoceros:,🦏
18,:lizard:,🦎
19,:gorilla:,🦍


In [9]:
import emoji
import json

# Create an emoji dictionary with labels and their Unicode values from https://carpedm20.github.io/emoji/docs/index.html
emoji_dict = {emoji.demojize(e): e for e in emoji.EMOJI_DATA}

# Show how many key value pairs we retrieved from emoji library
records = len(emoji_dict)
print(f"Number of records: {records}")

# Create new df for use in concatenation the two data frames
emoji_dictionary_df = pd.DataFrame(emoji_dict.items(), columns=['label', 'emoji'])
emoji_dictionary_df.head()
#emoji_dictionary_df.info()

Number of records: 3790


,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [10]:
# Concatentate both dataframes / sources and drop the index
combined_emoji_df = pd.concat([emoji_dictionary_df, new_filtered_df], ignore_index=True)

# Show the key pair data frame with training data
combined_emoji_df.head()

,label,emoji
0,:1st_place_medal:,🥇
1,:2nd_place_medal:,🥈
2,:3rd_place_medal:,🥉
3,:AB_button_(blood_type):,🆎
4,:ATM_sign:,🏧


In [26]:
# Remove : character used for outputting emoji (vs. unicode) and replace underscores with spaces
combined_emoji_df['label'] = combined_emoji_df['label'].str.replace(':', '').str.replace('_', ' ')

# Print output
combined_emoji_df.head(20)

,label,emoji
0,1st place medal,🥇
1,2nd place medal,🥈
2,3rd place medal,🥉
3,AB button (blood type),🆎
4,ATM sign,🏧
5,A button (blood type),🅰
6,Afghanistan,🇦🇫
7,Albania,🇦🇱
8,Algeria,🇩🇿
9,American Samoa,🇦🇸


In [28]:
# Convert DataFrame to a dictionary with label as the key
emoji_dict = combined_emoji_df.set_index('label')['emoji'].to_dict()

# Print the resulting dictionary
print(emoji_dict)

{'1st place medal': '🥇', '2nd place medal': '🥈', '3rd place medal': '🥉', 'AB button (blood type)': '🆎', 'ATM sign': '🏧', 'A button (blood type)': '🅰', 'Afghanistan': '🇦🇫', 'Albania': '🇦🇱', 'Algeria': '🇩🇿', 'American Samoa': '🇦🇸', 'Andorra': '🇦🇩', 'Angola': '🇦🇴', 'Anguilla': '🇦🇮', 'Antarctica': '🇦🇶', 'Antigua & Barbuda': '🇦🇬', 'Aquarius': '♒', 'Argentina': '🇦🇷', 'Aries': '♈', 'Armenia': '🇦🇲', 'Aruba': '🇦🇼', 'Ascension Island': '🇦🇨', 'Australia': '🇦🇺', 'Austria': '🇦🇹', 'Azerbaijan': '🇦🇿', 'BACK arrow': '🔙', 'B button (blood type)': '🅱', 'Bahamas': '🇧🇸', 'Bahrain': '🇧🇭', 'Bangladesh': '🇧🇩', 'Barbados': '🇧🇧', 'Belarus': '🇧🇾', 'Belgium': '🇧🇪', 'Belize': '🇧🇿', 'Benin': '🇧🇯', 'Bermuda': '🇧🇲', 'Bhutan': '🇧🇹', 'Bolivia': '🇧🇴', 'Bosnia & Herzegovina': '🇧🇦', 'Botswana': '🇧🇼', 'Bouvet Island': '🇧🇻', 'Brazil': '🇧🇷', 'British Indian Ocean Territory': '🇮🇴', 'British Virgin Islands': '🇻🇬', 'Brunei': '🇧🇳', 'Bulgaria': '🇧🇬', 'Burkina Faso': '🇧🇫', 'Burundi': '🇧🇮', 'CL button': '🆑', 'COOL button': '🆒', 'C

In [31]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

'''
# Define a dictionary mapping words to emojis
emoji_dict = {
    "happy": "😊",
    "sad": "😢",
    "love": "❤️",
    "dog": "🐶",
    "cat": "🐱",
    "apple": "🍏",
    "car": "🚗",
    "money": "💰",
    "party": "🎉",
    "coffee": "☕",
}
'''

# Prompt the user for input
user_input = input("Please enter a word: ")

# Process the input text with spaCy
doc = nlp(user_input)

# Initialize an empty list to hold the emojis
emojis = []

# Check for words in the user input and append corresponding emojis
for token in doc:
    if token.text.lower() in emoji_dict:
        emojis.append(emoji_dict[token.text.lower()])

# Print the corresponding emojis
if emojis:
    print("Corresponding Emojis:", " ".join(emojis))
else:
    print("No corresponding emojis found.")


Please enter a word: pig
Corresponding Emojis: 🐖
